# Bayesian Regression

Bayesian Resgression with rookies filtered out.

In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import arviz as az

data = pd.read_csv("../data/shifts_data_final_2018_19.csv")
data.drop(data.columns[0], axis = 1, inplace = True)
data.head()

,point_diff_per_100,home_team,away_team,0,1,2,3,4,5,6,...,519,520,521,522,523,524,525,526,527,528
0,-36.458333,Celtics,Nuggets,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,Celtics,Nuggets,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Note - the priors are too small. Scale them up to range from like -10 to 10

So far with standard dev of 4, the results are worse with these priors compared to the old priors we used before. We see random players with very high +/-

In [2]:
rookie_priors_df = pd.read_csv("../data/priors_rookies.csv")
rookie_priors_df.columns = ["index", "mu", "sd", "name", "type"]
rookie_priors_df

rookie_scale = 11.207822/max(rookie_priors_df.mu)

rookie_scale

rookie_priors_df.mu *= rookie_scale

rookie_priors_df

,index,mu,sd,name,type
0,1,9.005090,3,Jayson Tatum,Rookie
1,2,6.947821,3,Jaylen Brown,Rookie
2,3,4.099367,3,Terry Rozier,Rookie
3,14,2.624281,3,OG Anunoby,Rookie
4,16,3.409294,3,Delon Wright,Rookie
...,...,...,...,...,...
97,494,3.395777,3,Dario Saric,Rookie
98,504,2.569450,3,Aaron Holiday,Rookie
99,507,3.167747,3,Lonnie Walker IV,Rookie
100,514,7.736895,3,Brandon Ingram,Rookie


In [3]:
# store home and away teams for potential use later when we incorporate team ratings
home_teams = data['home_team']
away_teams = data['away_team']
# now drop these columns from the main training dataframe
data.drop(['home_team', 'away_team'], axis = 1, inplace = True)
data.head()

,point_diff_per_100,0,1,2,3,4,5,6,7,8,...,519,520,521,522,523,524,525,526,527,528
0,-36.458333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
prior_sd4 = np.full(101, 4)
prior_sd3 = np.full(101, 3)
prior_sd2 = np.full(101, 2)


idx_set = set(rookie_priors_df["index"].astype(str))

In [6]:
# need to rename columns now since numbers confuse pymc3

cols = [col for col in data.columns if col in idx_set or col == "point_diff_per_100"]

rookie_data = data[cols]
#rookie_data
rookie_data.head()

new_cols = []
for i in range(np.shape(rookie_data)[1]):
    if i == 0:
        new_cols.append("point_diff")
    else:
        new_cols.append("p" + rookie_data.columns[i])

x_df = rookie_data
x_df.columns = new_cols
x_df.head()


,point_diff,p1,p2,p3,p14,p16,p18,p24,p26,p33,...,p448,p458,p459,p472,p481,p494,p504,p507,p514,p515
0,-36.458333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
x = np.array(x_df.iloc[:,1:])
y = np.array(x_df.iloc[:,0])

x_shape = np.shape(x)[1]
    
with pm.Model() as model4:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = rookie_priors_df.mu # defined above
    x_prior_sigmas = rookie_priors_df.sd # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace4 = pm.sample(1000, tune = 1000, cores = 1)
    
    

/Users/reedpeterson/opt/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:468: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  FutureWarning,
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [x, Intercept, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 222 seconds.


## Below we save the trace

In [42]:
with model4:
    path = pm.save_trace(trace4, directory = "rookie_trace")
    print(path)

rookie_trace


## Below is the methodology for loading in the trace. Note - we need to load in all the training data again, and define the model, but we do not have to actually do the sampling we just load the trace.

In [44]:
with pm.Model() as loaded_model:
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = rookie_priors_df.mu # defined above
    x_prior_sigmas = rookie_priors_df.sd # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    loaded_trace = pm.load_trace("rookie_trace")
    
    loaded_results = az.summary(loaded_trace)

In [45]:
print((loaded_results.loc[loaded_results['mean'] > 5]).sort_values(by=['mean']))

         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
x[15]   5.273  2.154   1.322    9.383      0.035    0.026    3741.0    1349.0   
x[0]    5.308  1.904   1.707    8.824      0.036    0.026    2834.0    1432.0   
x[26]   5.360  1.867   1.908    8.874      0.036    0.027    2669.0    1581.0   
x[20]   5.687  1.809   2.108    8.716      0.033    0.026    3060.0    1312.0   
x[54]   6.293  2.032   2.685   10.360      0.037    0.028    3053.0    1507.0   
x[94]   6.631  1.665   3.372    9.681      0.028    0.022    3461.0    1443.0   
x[49]   6.940  2.384   2.420   11.236      0.042    0.032    3139.0    1357.0   
x[47]   7.401  2.582   2.613   12.087      0.042    0.032    3857.0    1526.0   
sigma  78.456  0.317  77.846   79.019      0.005    0.004    3900.0    1552.0   

       r_hat  
x[15]   1.00  
x[0]    1.00  
x[26]   1.00  
x[20]   1.00  
x[54]   1.00  
x[94]   1.00  
x[49]   1.00  
x[47]   1.00  
sigma   1.01  


## The following cells were for cross validating values of sigma. Not super useful now.

In [ ]:
with pm.Model() as model3:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = prior_means # defined above
    x_prior_sigmas = prior_sd3 # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace3 = pm.sample(1000, tune = 1000, cores = 1)

In [ ]:
with pm.Model() as model2:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = prior_means # defined above
    x_prior_sigmas = prior_sd2 # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace2 = pm.sample(1000, tune = 1000, cores = 1)

In [11]:
with model4:
    results_df4 = az.summary(trace4)
    
# with model3:
#     results_df3 = az.summary(trace3)
    
# with model2:
#     results_df2 = az.summary(trace2)

In [10]:
# import the player map dictionary to go between index, player_id, and name
player_index_map = pd.read_csv("../data/player_index_map.csv")

In [38]:
player_index_map.loc[player_index_map.index == 0]
rookie_priors_df.iloc[26]

index             124
mu            4.70332
sd                  3
name     Jamal Murray
type           Rookie
Name: 26, dtype: object

In [30]:
print((results_df4.loc[results_df4['mean'] > 5]).sort_values(by=['mean']))
results_df4

         mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
x[15]   5.273  2.154   1.322    9.383      0.035    0.026    3741.0    1349.0   
x[0]    5.308  1.904   1.707    8.824      0.036    0.026    2834.0    1432.0   
x[26]   5.360  1.867   1.908    8.874      0.036    0.027    2669.0    1581.0   
x[20]   5.687  1.809   2.108    8.716      0.033    0.026    3060.0    1312.0   
x[54]   6.293  2.032   2.685   10.360      0.037    0.028    3053.0    1507.0   
x[94]   6.631  1.665   3.372    9.681      0.028    0.022    3461.0    1443.0   
x[49]   6.940  2.384   2.420   11.236      0.042    0.032    3139.0    1357.0   
x[47]   7.401  2.582   2.613   12.087      0.042    0.032    3857.0    1526.0   
sigma  78.456  0.317  77.846   79.019      0.005    0.004    3900.0    1552.0   

       r_hat  
x[15]   1.00  
x[0]    1.00  
x[26]   1.00  
x[20]   1.00  
x[54]   1.00  
x[94]   1.00  
x[49]   1.00  
x[47]   1.00  
sigma   1.01  


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,1.863,0.426,1.008,2.587,0.008,0.006,3071.0,1647.0,1.00
x[0],5.308,1.904,1.707,8.824,0.036,0.026,2834.0,1432.0,1.00
x[1],-0.004,2.026,-3.588,3.892,0.039,0.045,2735.0,1554.0,1.00
x[2],-1.875,2.012,-5.890,1.705,0.039,0.034,2647.0,1417.0,1.00
x[3],1.409,2.035,-2.485,4.912,0.034,0.037,3514.0,1440.0,1.00
...,...,...,...,...,...,...,...,...,...
x[98],1.562,2.444,-2.977,5.893,0.036,0.049,4630.0,1591.0,1.00
x[99],1.767,2.775,-3.426,6.845,0.049,0.060,3218.0,1374.0,1.01
x[100],1.334,2.109,-2.922,5.058,0.037,0.039,3183.0,1523.0,1.00
x[101],2.597,2.330,-2.019,6.658,0.037,0.040,3991.0,1254.0,1.00


In [ ]:
print((results_df3.loc[results_df3['mean'] > 5]).sort_values(by=['mean']))

In [ ]:
print((results_df2.loc[results_df2['mean'] > 5]).sort_values(by=['mean']))

## The following cell shows how to get the distribution for a specific player

In this case we get the distribution for Steph Curry (index 317)

In [ ]:
import seaborn as sns

with model3:
#     az.plot_trace(trace3)
#     print(np.shape(trace3['x']))
#     print(max(trace3['x'][:,317]))
    sns.distplot(trace3['x'][:,317], hist = False)

In [ ]:
with model3:
    az.plot_trace(trace3)